<a href="https://colab.research.google.com/github/tytttyyyttyy/net_learning/blob/main/auto_cer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[record_sheet](https://docs.google.com/spreadsheets/d/1poMhLbeQSrMNnJgTW5zS1bi_hqf6PnHnEi_2aTJ8kRc/edit#gid=0)

[template_sheet
](https://docs.google.com/spreadsheets/d/1mQ2WUiGKHUJsQBuBJHzOhSst_lsIwYfmZE6y81bz2K4/edit?usp=drive_link)

#gradio


In [ ]:
!pip install gspread gradio openai

In [12]:
import pandas as pd
import gspread
import requests
from oauth2client.service_account import ServiceAccountCredentials
import gradio as gr

from google.colab import auth
auth.authenticate_user()

import os
import json
import openai
from openai import OpenAI

import googleapiclient.discovery
import google.auth
from google.auth.transport.requests import Request
from oauth2client.client import GoogleCredentials
from gspread import authorize


# 確保你已經將 Google 驗證憑證加載到環境中
creds, _ = google.auth.default()
gc = authorize(creds)

file_path = '/content/drive/My Drive/auto certificate/HR_Dataset_2024.csv'
df = pd.read_csv(file_path)  # 讀取 CSV 檔案

# 取得 Google Spreadsheets 工作表 : record_sheet
record_sheet_id = '1poMhLbeQSrMNnJgTW5zS1bi_hqf6PnHnEi_2aTJ8kRc'
record_sheet_name = 'record'
record_sheet = gc.open_by_key(record_sheet_id).worksheet(record_sheet_name)

def choose_empnum(emp_num):
    df['Emp #'] = df['Emp #'].astype(str)  # 將 Emp 欄位轉換為文字格式
    df_filtered = df[df['Emp #'] == emp_num]  # 篩選出符合條件的資料

    if len(df_filtered) > 0:
        employee_data = df_filtered[['Emp #', 'Employee Name', 'Location', 'National ID', 'Department', 'DOB', 'Position', 'Date of Termination', 'Gender']].to_dict(orient='records')[0]
        for key, value in employee_data.items():
            if pd.isna(value):
                employee_data[key] = 'nan'

        today = pd.Timestamp('today')  # 取得目前日期
        today_str = today.strftime('%Y-%m-%d')
        employee_data['Current Date'] = today_str  # 將目前日期加到字典中

        data = pd.DataFrame([employee_data]).values.tolist()  # 將DataFrame轉換為二維列表
        data = data[0]  # 將列表轉換為一維
        record_sheet.append_row(data)  # 將data導入record sheet

        return employee_data, data, emp_num

    else:
        return None, None, emp_num

def duplicate_sheet(spreadsheet_id, source_sheet_id, new_sheet_name):
    sheets_service = googleapiclient.discovery.build('sheets', 'v4')
    request = {
        'requests': [
            {
                'duplicateSheet': {
                    'sourceSheetId': source_sheet_id,
                    'newSheetName': new_sheet_name,
                }
            }
        ]
    }
    sheets_service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=request).execute()
    template_sheet = gc.open_by_key(spreadsheet_id).worksheet(new_sheet_name)
    return template_sheet

def data_to_copied(employee_data, data, template_sheet):
    employee_data_columns = pd.DataFrame([employee_data]).columns.tolist()
    a = pd.DataFrame([employee_data]).shape[1]  # a等於employee_data_columns數量

    for i in range(a):
        search_term = "{{ [" + employee_data_columns[i] + "] }}"
        replace_term = data[i]

        for row_index, row in enumerate(template_sheet.get_all_values()):
            for column_index, cell in enumerate(row):
                if search_term in cell:
                    search_term_column = column_index
                    search_term_row = row_index + 1
                    template_sheet.update_acell(f'{chr(ord("A") + search_term_column)}{search_term_row}', replace_term)

    copied_id = template_sheet.id
    return copied_id

def generate_pdf(template_sheet_id, copied_id, emp_num):
    pdf_url = f"https://docs.google.com/spreadsheets/d/{template_sheet_id}/export?format=pdf&gid={copied_id}"
    response = requests.get(pdf_url)
    pdf_file_name = f"termination_certificate_{emp_num}.pdf"
    pdf_file_path = "/content/drive/My Drive/auto certificate/outputs/" + pdf_file_name

    with open(pdf_file_path, 'wb') as f:
        f.write(response.content)
    return pdf_file_path

def delete_sheet(template_sheet_id, template_name):
    try:
        spreadsheet = gc.open_by_key(template_sheet_id)
        worksheet = spreadsheet.worksheet(template_name)
    except (KeyError, AttributeError) as e:
        print(f"Error accessing worksheet '{template_name}': {e}")
        return

    try:
        spreadsheet.del_worksheet(worksheet)
        print(f"Worksheet '{template_name}' deleted from spreadsheet with ID '{template_sheet_id}'")
    except Exception as e:
        print(f"Failed to delete worksheet '{template_name}': {e}")

def generate_termination_pdf(emp_num):
    template_sheet_id = '1mQ2WUiGKHUJsQBuBJHzOhSst_lsIwYfmZE6y81bz2K4'
    template_name = 'copied'
    employee_data, data, emp_num = choose_empnum(emp_num)

    if employee_data is None:
        return f"查無Emp # {emp_num} 的資料"

    template_sheet = duplicate_sheet(template_sheet_id, '0', template_name)
    copied_id = data_to_copied(employee_data, data, template_sheet)
    pdf_path = generate_pdf(template_sheet_id, copied_id, emp_num)
    delete_sheet(template_sheet_id, template_name)

    return f"PDF generated and saved at: {pdf_path}"

def call_openai(user_question):
    try:
        record_data = record_sheet.get_all_values()
        json_data = json.dumps(record_data)

        # from openai import OpenAI
        with open('/content/drive/My Drive/auto certificate/client/auto_cer_openai_apikey.txt', 'r') as f:
            api_key = f.read()

        os.environ["OPENAI_API_KEY"] = api_key
        # 初始化 OpenAI 客戶端

        client = openai.Client(api_key=os.environ["OPENAI_API_KEY"])

        completion = client.chat.completions.create(
          model="gpt-3.5-turbo",
          # 構建消息列表
          messages = [
              {"role": "system", "content": "You are a human resource secretary, please answer my questions based on information I provided."},
              {"role": "system", "content": "Please provide a precise answer based on the json_data file."},
              {"role": "system", "content": json_data},
              {"role": "user", "content": user_question}
          ]
        )
        print(completion.choices[0].message.content)
        return completion.choices[0].message.content
    except Exception as e:
        return f"Error occurred: {e}"

def main_generate_pdf(emp_num):
    return generate_termination_pdf(emp_num)

def main_openai_response(user_question):
    return call_openai(user_question)

def clear_question(question_input, openai_output):
  # Clear the question input field
  question_input.value = ""

  # Reset OpenAI response field
  openai_output.value = ""

with gr.Blocks() as demo:
  with gr.Tab("Generate PDF"):
      emp_num_input = gr.Textbox(label="Enter Employee Number")
      pdf_output = gr.Textbox(label="Generated PDF Path")
      generate_pdf_button = gr.Button("Generate PDF")
      generate_pdf_button.click(fn=main_generate_pdf, inputs=emp_num_input, outputs=pdf_output)

  with gr.Tab("Ask OpenAI"):
    question_input = gr.Textbox(label="Enter your question")
    openai_output = gr.Textbox(label="AI Response")
    ask_openai_button = gr.Button("Ask")
    #clear_question_button = gr.Button("Clear")

    ask_openai_button.click(fn=main_openai_response, inputs=question_input, outputs=openai_output)
    #clear_question_button.click(fn=clear_question, inputs=[question_input, openai_output])

demo.launch()



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0b8abd3ca08cced8c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
